Compute VAEP Values & Calculate Player Ratings

In [1]:
import sys
sys.path.append("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/")

In [2]:
import pandas as pd
import numpy as np
import joblib
from scipy import stats

import warnings
warnings.filterwarnings('ignore')

from config import chain_file_path
from vaep.domain.contracts.modelling_data_contract import ModellingDataContract
from vaep.domain.modelling.supermodel import SuperXGBClassifier
from vaep.domain.modelling.model_evaluation import XGBClassifierEvaluator
from vaep.domain.preprocessing.preprocessing import *
from vaep.domain.preprocessing.formula import *


pd.options.display.max_rows = 999
pd.options.display.max_columns = 999

%load_ext autoreload
%autoreload 2

Intel MKL WARNING: Support of Intel(R) Streaming SIMD Extensions 4.2 (Intel(R) SSE4.2) enabled only processors has been deprecated. Intel oneAPI Math Kernel Library 2025.0 will require Intel(R) Advanced Vector Extensions (Intel(R) AVX) instructions.


In [3]:
FEATURES = ModellingDataContract.feature_list_scores

Load chain data

In [4]:
chains = pd.read_csv(chain_file_path)
chains.head()

,Chain_Number,Initial_State,Final_State,Order,Quarter,Quarter_Duration_Chain_Start,Quarter_Duration,Team_Chain,Team,Player,AFL_API_Player_ID,Description,x,y,Disposal,Shot_At_Goal,Behind_Detail,Venue_Width,Venue_Length,Home_Team,Away_Team,Home_Team_Direction_Q1,Match_ID,Round_ID,Year,Season
0,1,centreBounce,goal,1.0,1,13,13.0,Brisbane Lions,NaN,NaN,NaN,Centre Bounce,0.0,0.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
1,1,centreBounce,goal,2.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Hard Ball Get,8.0,-5.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
2,1,centreBounce,goal,3.0,1,13,24.0,Brisbane Lions,Brisbane Lions,Dayne Zorko,Dayne_Zorko,Handball,9.0,-6.0,ineffective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
3,1,centreBounce,goal,4.0,1,13,28.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Loose Ball Get,11.0,-7.0,NaN,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN
4,1,centreBounce,goal,5.0,1,13,29.0,Brisbane Lions,Sydney,Oliver Florent,Oliver_Florent,Handball,12.0,-5.0,effective,NaN,NaN,138,156,Brisbane Lions,Sydney,right,202101_BrisbaneLions_Sydney,202101,2021.0,NaN


Preprocess

In [5]:
schema_chains = convert_chains_to_schema(chains)
gamestate_features = create_gamestate_features(schema_chains)

In [6]:
schema_chains.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective


In [9]:
vaep_modelling_data = pd.concat([schema_chains, gamestate_features], axis=1)
vaep_modelling_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,action_type_a0,type_Kick_a0,type_Handball_a0,type_Carry_a0,type_Uncontested Mark_a0,type_Contested Mark_a0,type_Loose Ball Get_a0,type_Hard Ball Get_a0,type_Spoil_a0,type_Gather_a0,type_Free For_a0,type_Knock On_a0,type_Shot_a0,type_Tackle_a0,type_Error_a0,outcome_type_a0,outcome_effective_a0,outcome_ineffective_a0,outcome_clanger_a0,type_Kick_outcome_effective_a0,type_Kick_outcome_ineffective_a0,type_Kick_outcome_clanger_a0,type_Handball_outcome_effective_a0,type_Handball_outcome_ineffective_a0,type_Handball_outcome_clanger_a0,type_Carry_outcome_effective_a0,type_Carry_outcome_ineffective_a0,type_Carry_outcome_clanger_a0,type_Uncontested Mark_outcome_effective_a0,type_Uncontested Mark_outcome_ineffective_a0,type_Uncontested Mark_outcome_clanger_a0,type_Contested Mark_outcome_effective_a0,type_Contested Mark_outcome_ineffective_a0,type_Contested Mark_outcome_clanger_a0,type_Loose Ball Get_outcome_effective_a0,type_Loose Ball Get_outcome_ineffective_a0,type_Loose Ball Get_outcome_clanger_a0,type_Hard Ball Get_outcome_effective_a0,type_Hard Ball Get_outcome_ineffective_a0,type_Hard Ball Get_outcome_clanger_a0,type_Spoil_outcome_effective_a0,type_Spoil_outcome_ineffective_a0,type_Spoil_outcome_clanger_a0,type_Gather_outcome_effective_a0,type_Gather_outcome_ineffective_a0,type_Gather_outcome_clanger_a0,type_Free For_outcome_effective_a0,type_Free For_outcome_ineffective_a0,type_Free For_outcome_clanger_a0,type_Knock On_outcome_effective_a0,type_Knock On_outcome_ineffective_a0,type_Knock On_outcome_clanger_a0,type_Shot_outcome_effective_a0,type_Shot_outcome_ineffective_a0,type_Shot_outcome_clanger_a0,type_Tackle_outcome_effective_a0,type_Tackle_outcome_ineffective_a0,type_Tackle_outcome_clanger_a0,type_Error_outcome_effective_a0,type_Error_outcome_ineffective_a0,type_Error_outcome_clanger_a0,quarter_a0,quarter_seconds_a0,overall_seconds_a0,start_x_a0,start_y_a0,end_x_a0,end_y_a0,dx_a0,dy_a0,movement_a0,action_type_a1,type_Kick_a1,type_Handball_a1,type_Carry_a1,type_Uncontested Mark_a1,type_Contested Mark_a1,type_Loose Ball Get_a1,type_Hard Ball Get_a1,type_Spoil_a1,type_Gather_a1,type_Free For_a1,type_Knock On_a1,type_Shot_a1,type_Tackle_a1,type_Error_a1,outcome_type_a1,outcome_effective_a1,outcome_ineffective_a1,outcome_clanger_a1,type_Kick_outcome_effective_a1,type_Kick_outcome_ineffective_a1,type_Kick_outcome_clanger_a1,type_Handball_outcome_effective_a1,type_Handball_outcome_ineffective_a1,type_Handball_outcome_clanger_a1,type_Carry_outcome_effective_a1,type_Carry_outcome_ineffective_a1,type_Carry_outcome_clanger_a1,type_Uncontested Mark_outcome_effective_a1,type_Uncontested Mark_outcome_ineffective_a1,type_Uncontested Mark_outcome_clanger_a1,type_Contested Mark_outcome_effective_a1,type_Contested Mark_outcome_ineffective_a1,type_Contested Mark_outcome_clanger_a1,type_Loose Ball Get_outcome_effective_a1,type_Loose Ball Get_outcome_ineffective_a1,type_Loose Ball Get_outcome_clanger_a1,type_Hard Ball Get_outcome_effective_a1,type_Hard Ball Get_outcome_ineffective_a1,type_Hard Ball Get_outcome_clanger_a1,type_Spoil_outcome_effective_a1,type_Spoil_outcome_ineffective_a1,type_Spoil_outcome_clanger_a1,type_Gather_outcome_effective_a1,type_Gather_outcome_ineffective_a1,type_Gather_outcome_clanger_a1,type_Free For_outcome_effective_a1,type_Free For_outcome_ineffective_a1,type_Free For_outcome_clanger_a1,type_Knock On_outcome_effective_a1,type_Knock On_outcome_ineffective_a1,type_Knock On_outcome_clanger_a1,type_Shot_outcome_effective_a1,type_Shot_outcome_ineffective_a1,type_Shot_outcome_clanger_a1,type_Tackle_outcome_effective_a1,type_Tackle_outcome_ineffective_a1,type_Tackle_outcome_clanger_a1,type_Error_outcome_effective_a1,type_Error_outcome_ineffective_a1,type_Error_outcome_clanger_a1,quarter_a1,quarter_seconds_a1,overall_seconds_a1,start_x_a1,start_y_a1,end_x_a1,end_y_a1,dx_a1,dy_a1,movement_a1,acti

In [8]:
vaep_features = vaep_modelling_data[FEATURES]

Load models

In [10]:
model_file_path = "/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/vaep/model_outputs/models/"
scoring_model = "vaep_scoring_v2.joblib"
conceding_model = "vaep_conceding_v2.joblib"

In [11]:
score_model = joblib.load(model_file_path + "/" + scoring_model)
concede_model = joblib.load(model_file_path + "/" + conceding_model)

Make predictions

In [12]:
schema_chains['scores'] = score_model.predict_proba(vaep_features, calibrate=True)
schema_chains['concedes'] = concede_model.predict_proba(vaep_features, calibrate=True)

Compute VAEP

In [13]:
def calculate_vaep_values(schema_chains):
    
    match_list = list(schema_chains['match_id'].unique())
    match_vaep_list = []
    for match in match_list:
        match_chains = schema_chains[schema_chains['match_id'] == match]
        v = value(match_chains, match_chains['scores'], match_chains['concedes'])
        match_vaep_list.append(v)
        
    vaep_values = pd.concat(match_vaep_list, axis=0)
    
    return pd.concat([schema_chains, vaep_values], axis=1)

    

In [14]:
match_list = list(schema_chains['match_id'].unique())
match_vaep_list = []
for match in match_list:
    match_chains = schema_chains[schema_chains['match_id'] == match]
    v = value(match_chains, match_chains['scores'], match_chains['concedes'])
    match_vaep_list.append(v)
    
vaep_values = pd.concat(match_vaep_list, axis=0)

In [15]:
chain_vaep_data = pd.concat([schema_chains, vaep_values], axis=1)
chain_vaep_data.head()

,match_id,chain_number,order,quarter,quarter_seconds,overall_seconds,team,player,start_x,start_y,end_x,end_y,action_type,outcome_type,scores,concedes,offensive_value,defensive_value,vaep_value
1,202101_BrisbaneLions_Sydney,1,2.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,8.0,-5.0,9.0,-6.0,Hard Ball Get,effective,0.059166,0.004202,0.000000,-0.000000,0.000000
2,202101_BrisbaneLions_Sydney,1,3.0,1,24.0,24.0,Brisbane Lions,Dayne Zorko,9.0,-6.0,-11.0,7.0,Handball,ineffective,0.013482,0.020271,-0.045684,-0.016070,-0.061754
3,202101_BrisbaneLions_Sydney,1,4.0,1,28.0,28.0,Sydney,Oliver Florent,-11.0,7.0,-12.0,5.0,Loose Ball Get,effective,0.041042,0.016021,0.020770,-0.002539,0.018231
4,202101_BrisbaneLions_Sydney,1,5.0,1,29.0,29.0,Sydney,Oliver Florent,-12.0,5.0,-14.0,2.0,Handball,effective,0.028680,0.018624,-0.012362,-0.002603,-0.014965
5,202101_BrisbaneLions_Sydney,1,6.0,1,30.0,30.0,Sydney,George Hewett,-14.0,2.0,-22.0,2.0,Carry,effective,0.032836,0.015452,0.004157,0.003172,0.007328


In [ ]:
chain_vaep_data.to_csv("/Users/ciaran/Documents/Projects/AFL/git-repositories/afl-player-ratings/data/schema_chains_vaep_values.csv", index=False)